In [7]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix
        self.rows = len(matrix)
        self.cols = len(matrix[0])

    def display(self):
        print("Matrix:")
        for row in self.matrix:
            print(row)

    def transpose(self):

        result = []
        for i in range(self.cols):
            new_row = []
            for j in range(self.rows):
                new_row.append(self.matrix[j][i])
            result.append(new_row)
        return Matrix(result)

    def echelon_form(self):

        matrix_copy = []
        for row in self.matrix:
          matrix_copy.append(row[:])
        n = self.rows
        num_row_swaps = 0

        for i in range(n):

            pivot = matrix_copy[i][i]
            if pivot == 0:

                for k in range(i + 1, n):
                    if matrix_copy[k][i] != 0:
                        matrix_copy[i], matrix_copy[k] = matrix_copy[k], matrix_copy[i]
                        num_row_swaps += 1
                        pivot = matrix_copy[i][i]
                        break
                if pivot == 0:
                    return matrix_copy, num_row_swaps


            for j in range(i + 1, n):
                ratio = matrix_copy[j][i] / pivot
                for k in range(i, n):
                    matrix_copy[j][k] -= ratio * matrix_copy[i][k]

        return matrix_copy, num_row_swaps

    def determinant(self):
        if self.rows != self.cols:
            print("Determinant is only defined for square matrices")
            return None
        echelon_matrix, row_swaps = self.echelon_form()
        det = 1
        for i in range(self.rows):
            det *= echelon_matrix[i][i]
        if row_swaps % 2 != 0:
            det *= -1
        return det


    def rank(self):

        echelon_matrix,_ = self.echelon_form()
        rank = 0

        # Count the number of non-zero rows
        for row in echelon_matrix:
            if any(element != 0 for element in row):
                rank += 1
        return rank

    def minor(self, i, j):
        if self.rows != self.cols:
            print("Minor is only defined for square matrices")
            return None

        minor_matrix = []
        for row_index in range(self.rows):
            if row_index != i:
                new_row = []
                for col_index in range(self.cols):
                    if col_index != j:
                        new_row.append(self.matrix[row_index][col_index])
                minor_matrix.append(new_row)
        return Matrix(minor_matrix)

    def cofactor_matrix(self):
        if self.rows != self.cols:
            print("Cofactor matrix is only defined for square matrices")
            return None
        cofactors = []
        for i in range(self.rows):
            row_cofactors = []
            for j in range(self.cols):
                minor_det = self.minor(i, j).determinant()
                cofactor = ((-1) ** (i + j)) * minor_det
                row_cofactors.append(cofactor)
            cofactors.append(row_cofactors)
        return Matrix(cofactors)

    def adjoint(self):
        if self.rows != self.cols:
            print("Adjoint is only defined for square matrices")
            return None
        cofactor_matrix = self.cofactor_matrix()
        adjoint_matrix = cofactor_matrix.transpose()
        return adjoint_matrix

    def inverse(self):
        if self.rows != self.cols:
            print("Inverse is only defined for square matrices")
            return None
        det = self.determinant()
        if det == 0:
            print("Matrix is singular and cannot be inverted!")
            return None
        adjoint_matrix = self.adjoint()
        inverse_matrix = []
        for i in range(self.rows):
            new_row = []
            for j in range(self.cols):
                new_row.append(adjoint_matrix.matrix[i][j] / det)
            inverse_matrix.append(new_row)
        return Matrix(inverse_matrix)

    def solve_cramers_rule(self, b):
        if self.rows != self.cols:
            print("Cramer's Rule can only be applied to square matrices.")
            return None

        if len(b) != self.rows:
            print("Dimension of vector b must be the same as the number of rows in the matrix.")
            return None
        det_A = self.determinant()
        if det_A == 0:
            print("The system has no unique solution since the determinant is zero.")
            return None
        x = [0] * self.rows

        for i in range(self.cols):
            # Create a new matrix A_i by replacing the i-th column of A with the vector b
            A_i = []
            for row_index in range(self.rows):
                new_row = []
                for col_index in range(self.cols):
                    if col_index == i:
                        new_row.append(b[row_index])  # Replace the column with b
                    else:
                        new_row.append(self.matrix[row_index][col_index])
                A_i.append(new_row)

            matrix_A_i = Matrix(A_i)
            det_A_i = matrix_A_i.determinant()
            x[i] = det_A_i / det_A
        return x

    def augmented_matrix(self, b):
        augmented = []
        for i in range(self.rows):
            new_row = self.matrix[i][:]
            new_row.append(b[i])
            augmented.append(new_row)
        return Matrix(augmented)

    def solve_using_rank(self, b):
        if len(b) != self.rows:
            print("Dimension mismatch: b must have the same number of rows as the matrix A.")
            return None

        rank_A = self.rank()
        augmented = self.augmented_matrix(b)
        rank_augmented = augmented.rank()

        if rank_A != rank_augmented:
            return "No solution"
        elif rank_A == rank_augmented == self.cols:
            return self.solve_cramers_rule(b)
        else:
            return "Infinite solutions"

    def multiply(self, other):
        if self.cols != other.rows:
            print("Matrix multiplication is not possible. Number of columns in the first matrix must equal the number of rows in the second matrix.")
            return None
        result = []
        for i in range(self.rows):
            new_row = []
            for j in range(other.cols):
                new_row.append(0)
            result.append(new_row)
        for i in range(self.rows):
            for j in range(other.cols):
                for k in range(self.cols):
                    result[i][j] += self.matrix[i][k] * other.matrix[k][j]
        return Matrix(result)

    def is_identity(self):
        if self.rows != self.cols:
            return False
        for i in range(self.rows):
            for j in range(self.cols):
                if i == j and self.matrix[i][j] != 1:
                    return False
                elif i != j and self.matrix[i][j] != 0:
                    return False
        return True

    def is_symmetric(self):
        return self.matrix == self.transpose().matrix

    def is_skew_symmetric(self):
        transpose_matrix = self.transpose().matrix
        for i in range(self.rows):
            for j in range(self.cols):
                if self.matrix[i][j] != -transpose_matrix[i][j]:
                    return False
        return True

    def is_orthogonal(self):
        transpose_matrix = self.transpose()
        product_matrix = self.multiply(transpose_matrix)
        return product_matrix.is_identity()

    def is_idempotent(self):
        product_matrix = self.multiply(self)
        return product_matrix.matrix == self.matrix

    def is_involutory(self):
        product_matrix = self.multiply(self)
        return product_matrix.is_identity()

    def eigenvalues(self):
        if self.rows != 2 or self.cols != 2:
            print("Eigenvalues computation is currently implemented only for 2x2 matrices.")
            return None

        a, b = self.matrix[0][0], self.matrix[0][1]
        c, d = self.matrix[1][0], self.matrix[1][1]

        trace = a + d
        determinant = a * d - b * c
        discriminant = trace ** 2 - 4 * determinant

        if discriminant < 0:
            real_part = trace / 2
            imaginary_part = (-discriminant)**0.5 / 2
            return (complex(real_part, imaginary_part), complex(real_part, -imaginary_part))

        lambda1 = (trace + ((discriminant)**0.5)) / 2
        lambda2 = (trace - ((discriminant)**0.5)) / 2

        return (lambda1, lambda2)



In [8]:
m1 = Matrix([[1, 2, 3], [4, 5, 6]])
m2 = Matrix([[2,1,0],[-1,1,9],[-5,0,3]])
m2.display()
det=m2.determinant()
print('\nDeterminant',end=" ")
print(det)

print("\nEchelon Form:")
echelon_matrix,n = m2.echelon_form()
for row in echelon_matrix:
    print(row)



rank_val = m2.rank()
print(f"\nRank: {rank_val}")
print("\nInverse Matrix:")
inverse_matrix = m2.inverse()
print("\n")
inverse_matrix.display()

m3 = Matrix([[2, 1, -1], [-3, -1, 2], [-2, 1, 2]])
b = [8, -11, -3]
print('\n')
m3.display()
solution = m3.solve_cramers_rule(b)

print("\nSolution:")
print(solution)


m4 = Matrix([[0, -2], [2, 0]])
print('\n')
m4.display()
print("\nSkew-Symmetric:", m4.is_skew_symmetric())
print("\nSymmetric:", m4.is_symmetric())
print("\nOrthogonal:", m4.is_orthogonal())
print("\nIdempotent:", m4.is_idempotent())
print("\nInvolutory:", m4.is_involutory())

m5 = Matrix([[4, 2], [1, 3]])
print('\n')
m5.display()
print("\nEigenvalues:", m5.eigenvalues())

Matrix:
[2, 1, 0]
[-1, 1, 9]
[-5, 0, 3]

Determinant -36.0

Echelon Form:
[2, 1, 0]
[0.0, 1.5, 9.0]
[0.0, 0.0, -12.0]

Rank: 3

Inverse Matrix:


Matrix:
[-0.08333333333333333, 0.08333333333333333, -0.25]
[1.1666666666666667, -0.16666666666666666, 0.5]
[-0.1388888888888889, 0.1388888888888889, -0.08333333333333333]


Matrix:
[2, 1, -1]
[-3, -1, 2]
[-2, 1, 2]

Solution:
[1.9999999999999996, 3.0, -1.0]


Matrix:
[0, -2]
[2, 0]

Skew-Symmetric: True

Symmetric: False

Orthogonal: False

Idempotent: False

Involutory: False


Matrix:
[4, 2]
[1, 3]

Eigenvalues: (5.0, 2.0)


In [9]:
m = Matrix([[0,0],[0,0]])
det = m.determinant()
print(det)

0


In [10]:
A = Matrix([[2, 1, -1], [3, 3, 9]])
b = [8, 0]

solution = A.solve_using_rank(b)
print("Solution:", solution)

Solution: Infinite solutions
